In [1]:
data_path = "/scratch/cl6707/Shared_Datasets/NSD_MindEye"
nsd_root = "/scratch/cl6707/Projects/neuro_interp/data/NSD/"
stim_root = nsd_root + "nsddata_stimuli/stimuli/nsd/"
beta_root = nsd_root + "nsddata_betas/ppdata/"
mask_root = nsd_root + "nsddata/ppdata/"
nsd_mindroot = '/scratch/cl6707/Shared_Datasets/NSD_MindEye'
subj = 1
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
import sys
sys.path.append('/scratch/ne2213/projects/tmp_packages')
sys.path.append('/scratch/ne2213/projects/tmp_packages/')
sys.path.append('/scratch/cl6707/Projects/neuro_interp/data/NSD/nsd')
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
local_rank = 0
print("device:",device)

import utils

if utils.is_interactive():
    %load_ext autoreload
    %autoreload 2

seed=42
utils.seed_everything(seed=seed)
from src.file_utility import load_mask_from_nii, view_data

%matplotlib inline
fpX = np.float32
import src.numpy_utility as pnu
from src.file_utility import load_mask_from_nii
from nilearn.decoding import SpaceNetRegressor
import nibabel as nib

subj =1
voxel_roi_full  = load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/prf-visualrois.nii.gz"%subj)
voxel_roi_full.shape

# from NSDAccess import NSDAccess
# nsd = NSDAccess('/scratch/cl6707/Projects/neuro_interp/data/NSD/')

if subj == 1:
    num_voxels = 15724
elif subj == 2:
    num_voxels = 14278
elif subj == 3:
    num_voxels = 15226
elif subj == 4:
    num_voxels = 13153
elif subj == 5:
    num_voxels = 13039
elif subj == 6:
    num_voxels = 17907
elif subj == 7:
    num_voxels = 12682
elif subj == 8:
    num_voxels = 14386
    
    
print('Pulling NSD webdataset data...')
# Multi-GPU config #
from accelerate import Accelerator
accelerator = Accelerator(split_batches=False,mixed_precision='fp16')  
print("PID of this process =",os.getpid())
print = accelerator.print # only print if local_rank=0
device = accelerator.device
print("device:",device)
num_devices = torch.cuda.device_count()
if num_devices==0: num_devices = 1
num_workers = num_devices
print(accelerator.state)
local_rank = accelerator.state.local_process_index
world_size = accelerator.state.num_processes
distributed = not accelerator.state.distributed_type == 'NO'
print("distributed =",distributed, "num_devices =", num_devices, "local rank =", local_rank, "world size =", world_size)



# train_url = "{" + f"{data_path}/webdataset_avg_split/train/train_subj0{subj}_" + "{0..17}.tar," + f"{data_path}/webdataset_avg_split/val/val_subj0{subj}_0.tar" + "}"
# val_url = f"{data_path}/webdataset_avg_split/test/test_subj0{subj}_" + "{0..1}.tar"
# print(train_url,"\n",val_url)
# meta_url = f"{data_path}/webdataset_avg_split/metadata_subj0{subj}.json"
# num_train = 8559 + 300
# num_val = 982
# batch_size = 1
# print('Prepping train and validation dataloaders...')

# train_dl, val_dl, num_train, num_val = utils.get_dataloaders(
#     batch_size,'images',
#     num_devices=num_devices,
#     num_workers=num_workers,
#     train_url=train_url,
#     val_url=val_url,
#     meta_url=meta_url,
#     num_train=num_train,
#     num_val=num_val,
#     val_batch_size=1,
#     cache_dir=data_path, #"/tmp/wds-cache",
#     seed=seed,
#     voxels_key='nsdgeneral.npy', # 'nsdgeneral.npy' (1d), 'wholebrain_3d.npy'(3d)
#     to_tuple=["voxels", "images", "coco","trial"],
#     local_rank=local_rank,
#     world_size=world_size,
#     reloading=True,
    
# )

device: cpu


2023-12-24 19:52:57.326280: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-24 19:52:57.869284: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-24 19:52:58.136098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 19:52:58.136124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 19:52:58.147232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Pulling NSD webdataset data...
PID of this process = 840560
device: cpu
Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cpu

Mixed precision type: fp16

distributed = False num_devices = 1 local rank = 0 world size = 1


In [5]:
nsdgeneral_affine = nib.load('/scratch/cl6707/Projects/neuro_interp/data/NSD/nsddata/ppdata/subj01/func1pt8mm/roi/nsdgeneral.nii.gz').affine
nsdgeneral =  nib.load('/scratch/cl6707/Projects/neuro_interp/data/NSD/nsddata/ppdata/subj01/func1pt8mm/roi/nsdgeneral.nii.gz')
nsdgeneral_roi_mask = (nsdgeneral.get_fdata()==1).astype(np.float32)
nsdgeneral = nib.Nifti1Image(nsdgeneral_roi_mask, nsdgeneral_affine)
anat_img = '/scratch/cl6707/Projects/neuro_interp/data/NSD/nsddata/ppdata/subj01/func1pt8mm/T1_to_func1pt8mm.nii.gz'
things = np.load(nsd_mindroot + '/subj%02d_things.npy'%subj,allow_pickle=True)
things_all = np.concatenate(things, axis=0)
unique_things = np.unique(things_all)
things_val_mapping = {k:i for i,k in enumerate(unique_things)}
val_things_mapping = {i:k for i,k in enumerate(unique_things)}


def reconstruct_volume_corrected(vol_shape, binary_mask, data_vol, order='C'):
    
    view_vol = np.ones(np.prod(vol_shape), dtype=np.float32) * np.nan
    
    idx_mask = np.where(binary_mask)[0]
    
    view_vol[idx_mask] = data_vol
    return np.nan_to_num(view_vol.reshape(vol_shape, order=order))

file_path = '/scratch/ne2213/projects/IVP/Neural_Interpretation-main/notebooks/new_dataloader_50.pkl'
file_path_val = '/scratch/ne2213/projects/IVP/Neural_Interpretation-main/notebooks/feature_files/val_dataloader_50.pkl'
data_loader_pca = pickle.load(open(file_path, 'rb'))
data_loader_pca_val = pickle.load(open(file_path_val, 'rb'))


In [15]:


train_iter = iter(data_loader_pca)
val_iter = iter(data_loader_pca_val)
X_train_all = []
y_train_all = []
X_val_all = []
y_val_all = []

for i in tqdm(range(num_train)):
    voxels, images, coco, trial, feature = next(train_iter)
    voxels = voxels[0].cpu().numpy().mean(axis=0)
    # convert voxels to 3d volume
    voxels = reconstruct_volume_corrected(nsdgeneral_roi_mask.shape, nsdgeneral_roi_mask.flatten(), voxels)
    nii_voxels = nib.Nifti1Image(voxels, nsdgeneral_affine)
    labels = things[trial]
    # for label in labels:
    #     X_train_all.append(nii_voxels)
    #     y_train_all.append(things_val_mapping[label])
    if len(labels)>=1 and len(labels)<=3:
        X_train_all.append(feature.flatten().cpu().numpy())
        y_train_all.append(things_val_mapping[labels[0]])

for i in tqdm(range(num_val)):
    voxels, images, coco, trial, feature = next(val_iter)
    voxels = voxels[0].cpu().numpy().mean(axis=0)
    # convert voxels to 3d volume
    voxels = reconstruct_volume_corrected(nsdgeneral_roi_mask.shape, nsdgeneral_roi_mask.flatten(), voxels)
    nii_voxels = nib.Nifti1Image(voxels, nsdgeneral_affine)
    labels = things[trial]
    # for label in labels:
    #     X_val_all.append(nii_voxels)
    #     y_val_all.append(things_val_mapping[label])
    if len(labels)>=1 and len(labels)<=3:
        X_val_all.append(feature.flatten().cpu().numpy())
        y_val_all.append(things_val_mapping[labels[0]])

  0%|          | 0/8859 [00:00<?, ?it/s]

  0%|          | 0/982 [00:00<?, ?it/s]

In [12]:
voxels, images, coco, trial = next(iter(train_dl))


In [53]:
import h5py
with h5py.File('/scratch/cl6707/Shared_Datasets/NSD_MindEye/subj01_nsdgeneral.hdf5', 'r') as f:
    for key in f.keys():
        print(f[key].shape)

(27750, 256, 256, 3)
(27750, 15724)


In [13]:
import json
subj=2
meta_url = f"{data_path}/webdataset_avg_split/metadata_subj0{subj}.json"
print(meta_url)
meta_file = json.load(open(meta_url, 'r'))
meta_file

/scratch/cl6707/Shared_Datasets/NSD_MindEye/webdataset_avg_split/metadata_subj02.json


{'totals': {'train': 8559, 'val': 300, 'test': 982}}

{'totals': {'train': 8559, 'val': 300, 'test': 982}}

In [14]:

resnet50 = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
#get the last layer of the resnet50
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))
resnet50.eval().to(device)
for subj in [2,5,7]:
    train_url = "{" + f"{data_path}/webdataset_avg_split/train/train_subj0{subj}_" + "{0..17}.tar," + f"{data_path}/webdataset_avg_split/val/val_subj0{subj}_0.tar" + "}"
    val_url = f"{data_path}/webdataset_avg_split/test/test_subj0{subj}_" + "{0..1}.tar"
    print(train_url,"\n",val_url)
    meta_url = f"{data_path}/webdataset_avg_split/metadata_subj0{subj}.json"
    num_train = 8559 + 300
    num_val = 982
    batch_size = 1
    print('Prepping train and validation dataloaders...')

    train_dl, val_dl, num_train, num_val = utils.get_dataloaders(
        batch_size,'images',
        num_devices=num_devices,
        num_workers=num_workers,
        train_url=train_url,
        val_url=val_url,
        meta_url=meta_url,
        num_train=num_train,
        num_val=num_val,
        val_batch_size=1,
        cache_dir=data_path, #"/tmp/wds-cache",
        seed=seed,
        voxels_key='nsdgeneral.npy', # 'nsdgeneral.npy' (1d), 'wholebrain_3d.npy'(3d)
        to_tuple=["voxels", "images", "coco","trial"],
        local_rank=local_rank,
        world_size=world_size,
        reloading=True,
        
    )
    
    voxels_train_all = []
    image_train_all = []
    coco_train_all = []
    trial_train_all = []
    feature_train_all = []
    train_iter = iter(train_dl)
    for _ in tqdm(range(num_train)):
        voxels, images, coco, trial =  next(train_iter)
        voxels_train_all.append(voxels.cpu().numpy())
        image_train_all.append(images.cpu().numpy())
        coco_train_all.append(coco.cpu().numpy())
        trial_train_all.append(trial.cpu().numpy())


    voxels_val_all = []
    image_val_all = []
    coco_val_all = []
    trial_val_all = []
    feature_val_all = []
    val_iter = iter(val_dl)
    for _ in tqdm(range(num_val)):
        voxels, images, coco, trial =  next(val_iter)
        voxels_val_all.append(voxels.cpu().numpy())
        image_val_all.append(images.cpu().numpy())
        coco_val_all.append(coco.cpu().numpy())
        trial_val_all.append(trial.cpu().numpy())

        
    voxels_train_all = np.concatenate(voxels_train_all, axis=0)
    image_train_all = np.concatenate(image_train_all, axis=0)
    coco_train_all = np.concatenate(coco_train_all, axis=0)
    trial_train_all = np.concatenate(trial_train_all, axis=0)


    voxels_val_all = np.concatenate(voxels_val_all, axis=0)
    image_val_all = np.concatenate(image_val_all, axis=0)
    coco_val_all = np.concatenate(coco_val_all, axis=0)
    trial_val_all = np.concatenate(trial_val_all, axis=0)

    for image in tqdm(image_train_all):
        image = torch.from_numpy(image).to(device).unsqueeze(0)
        feature_train_all.append(resnet50(image).cpu().detach().numpy())
    for image in tqdm(image_val_all):
        image = torch.from_numpy(image).to(device).unsqueeze(0)
        feature_val_all.append(resnet50(image).cpu().detach().numpy())

    feature_train_all = np.concatenate(feature_train_all, axis=0)
    feature_val_all = np.concatenate(feature_val_all, axis=0)

    from sklearn.decomposition import PCA
    pca50= PCA(n_components=50)
    feature_all = np.concatenate([feature_train_all, feature_val_all], axis=0).squeeze()
    pca50.fit(feature_all)
    pca_feature_train_all = pca50.transform(feature_train_all.squeeze())
    pca_feature_val_all = pca50.transform(feature_val_all.squeeze())

    import h5py
    nsd_path = '/scratch/cl6707/Shared_Datasets/NSD_MindEye/'
    # Save the data to a hdf5 file

    with h5py.File(nsd_path + 'train_subj%02d_pca50.hdf5'%subj, 'w') as f:
        f.create_dataset('voxels', data=voxels_train_all)
        f.create_dataset('images', data=image_train_all)
        f.create_dataset('coco', data=coco_train_all)
        f.create_dataset('trial', data=trial_train_all)
        f.create_dataset('res_feature', data=feature_train_all)
        f.create_dataset('pca_feature', data=pca_feature_train_all)

    with h5py.File(nsd_path + 'val_subj%02d_pca50.hdf5'%subj, 'w') as f:
        f.create_dataset('voxels', data=voxels_val_all)
        f.create_dataset('images', data=image_val_all)
        f.create_dataset('coco', data=coco_val_all)
        f.create_dataset('trial', data=trial_val_all)
        f.create_dataset('res_feature', data=feature_val_all)
        f.create_dataset('pca_feature', data=pca_feature_val_all)

Using cache found in /home/cl6707/.cache/torch/hub/pytorch_vision_v0.6.0
/ext3/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/ext3/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


{/scratch/cl6707/Shared_Datasets/NSD_MindEye/webdataset_avg_split/train/train_subj02_{0..17}.tar,/scratch/cl6707/Shared_Datasets/NSD_MindEye/webdataset_avg_split/val/val_subj02_0.tar} 
 /scratch/cl6707/Shared_Datasets/NSD_MindEye/webdataset_avg_split/test/test_subj02_{0..1}.tar
Prepping train and validation dataloaders...
Getting dataloaders...

num_train 8859
global_batch_size 1
batch_size 1
num_workers 1
num_batches 8859
num_worker_batches 8859
Reloading Training data without shuffling!

num_val 982
val_num_batches 982
val_batch_size 1


  0%|          | 0/8859 [00:00<?, ?it/s]

  0%|          | 0/982 [00:00<?, ?it/s]

  0%|          | 0/8859 [00:00<?, ?it/s]

...

{/scratch/cl6707/Shared_Datasets/NSD_MindEye/webdataset_avg_split/train/train_subj07_{0..17}.tar,/scratch/cl6707/Shared_Datasets/NSD_MindEye/webdataset_avg_split/val/val_subj07_0.tar} 
 /scratch/cl6707/Shared_Datasets/NSD_MindEye/webdataset_avg_split/test/test_subj07_{0..1}.tar
Prepping train and validation dataloaders...
Getting dataloaders...

num_train 8859
global_batch_size 1
batch_size 1
num_workers 1
num_batches 8859
num_worker_batches 8859
Reloading Training data without shuffling!

num_val 982
val_num_batches 982
val_batch_size 1


  0%|          | 0/8859 [00:00<?, ?it/s]

  0%|          | 0/982 [00:00<?, ?it/s]

  0%|          | 0/8859 [00:00<?, ?it/s]

  0%|          | 0/982 [00:00<?, ?it/s]